In [1]:
#Universidad Nacional Autónoma de México
#Posgrado en Ciencias de la Tierra 
#Campo 1, Geofísica de la Tierra sólida, Sismología

#Elaborado por M.C. Isaías Manuel Ramírez Bañales e Isaac Valverde, 2023

#Programa 4_1: Notebook de Python 3 que calcula muestras de la distribución posterior del problema inverso

In [2]:
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import time

In [3]:
#Función elaborada por el M.C. Isaías Manuel Ramírez Bañales
def rho(z,L,U):
    if L<0 and U>0:        
        r=np.exp(-z**2/2)
    elif U<0:        
        r=np.exp((U**2-z**2)/2)

    elif L>0:        
        r=np.exp((L**2-z**2)/2)
    return r

In [4]:
#Función elaborada por el M.C. Isaías Manuel Ramírez Bañales
def SimulaEstandarTruncada(L,U):
    flag=0
    while flag==0:
        z=np.random.uniform(low=L,high=U,size=1)
        
        if np.random.uniform(size=1)<rho(z,L,U):
            x=z
            flag=1
    return x

In [5]:
#Función elaborada por el M.C. Isaías Manuel Ramírez Bañales
def Coef(i,Mu,Evalua,Directorio,Directorio2):
    mu1=np.delete(Mu,i)
    theta1=np.delete(Evalua,i)    
    mureduc=Mu[i]+Directorio[i]@(theta1-mu1)
    sigma2reduc=Directorio2[i]
    return np.array((mureduc,sigma2reduc[0]))

In [6]:
#Función elaborada por el M.C. Isaías Manuel Ramírez Bañales
def SimulaTruncada(Mu,Var,L,U,SampleSize):
    dim=len(Mu)
    InvVar=np.linalg.inv(Var)
    auxIZ=0

    for i in range(dim):
        Sigma1=np.delete(Var[:,i],i,axis=0)    
        Sigma1=np.reshape(Sigma1,(len(Sigma1),1))
        SInvII=np.delete(InvVar,i,axis=0)
        SInvII=np.delete(SInvII,i,axis=1)
        SInvIII=np.delete(InvVar[:,i],i,axis=0)
        ######
        SInvIII=np.reshape(SInvIII,(len(Sigma1),1))
        ######
    
        InversaReducida=SInvII-SInvIII@SInvIII.T/InvVar[i,i]
        
        M1=Sigma1.T@InversaReducida
        sigma2reduc=Var[i,i]-M1@Sigma1
        
        if i==0:
            Directorio=[M1]
            Directorio2=[sigma2reduc]
        else :
            Directorio.append(M1)
            Directorio2.append(sigma2reduc)
    
    #######################    
    Inicial=np.ones(dim)
    Inicial=np.reshape(Inicial,(1,dim))
    for i in range(SampleSize):
        Momentos=Coef(i%dim,Mu, Inicial[-1,:],Directorio,Directorio2)
        Lp,Up=(np.array((L,U))-Momentos[0])/np.sqrt(Momentos[1])
        Propuesta=np.copy(Inicial[-1,:])
        Propuesta[i%dim]=SimulaEstandarTruncada(Lp,Up)*np.sqrt(Momentos[1])+Momentos[0]
        Inicial=np.vstack((Inicial,Propuesta))
        if ((i%10000)==0 and i>100):
            auxIZ=auxIZ+1
            np.savetxt("MuestrasCrudo_a"+str(auxIZ)+".txt",Inicial[-10000:,:])
            print((i/SampleSize)*100,"%")
                
    return Inicial

In [ ]:
#Funcion que calcula el vector de medias y la matriz de covarianza de la función de densidad de probabilidad posterior a partir de las expresiones de Tarantola y Valette (1982)
#Entradas
#G: Matriz kernel de la inversión (dimensión número de datos por número de parámetros) 
#d: Vector columna que contiene los datos de desplazamiento (de dimensión 1 por número de datos)
#Cdinv: Matriz de covarianza de los datos
#alpha:Valor de alpha para aproximar la distribución apriori
#limsup: Vector que contiene el límite superior del intervalo de truncamiento de cada parámetro
#m0: Media de la distribucion apriori
#Salida
#mv: Vector de medias de la posterior
#Cmv: Matriz de covarianza de la posterior

def mvCmv(G,d,Cdinv,alpha,limsup,m0):
    
    #Se obtiene el número de parámetros
    Nm=np.shape(G)[1]
    #Se obtiene la transpuesta del kernel G
    GT=np.transpose(G)
    #Se define la desviación estandar (pp 373, primer párrafo, quinto renglón)
    sigma=alpha*(limsup/2)
    #Se define la matriz de covarianza de los parámetros (pp. 372, primer parrafo, segundo renglón)
    Cm=np.identity(Nm)*(((np.ones(Nm)*sigma).reshape(Nm,1))**2)
    #Se define la inversa la matriz de covarianza de los parámetros
    Cminv=np.linalg.inv(Cm)
    #Se define la matriz de covarianza posterior (Ecuaciones de Tarantola y Valette, pp. 369, Ec. 7)
    Cmv=np.linalg.inv(np.matmul(np.matmul(GT,Cdinv),G)+Cminv)
    #Se define la transpuesta de la matriz de covarianza posterior
    CmvT=np.transpose(Cmv)
    #Se define la inversa de la matriz de covarianza posterior
    Cmvinv=np.matmul(np.matmul(GT,Cdinv),G)+Cminv
    #Se define la expectativa (Ecuaciones de Tarantola y Valette, pp. 369, Ec. 6)
    mv=np.matmul(np.linalg.inv(np.matmul(np.matmul(GT,Cdinv),G)+Cminv), np.matmul(np.matmul(GT,Cdinv),d)+np.matmul(Cminv,m0))
    return mv, Cmv

In [8]:
#Se define el número de datos (3 componentes por 11 estaciones)
Nd=33
#Se define el número de parámetros (Discretización de falla de 18x18)
Nm=324
#Discretización de cada parámetro 
Ndm=100

In [9]:
#----------------------------Lectura y asignación de datos----------------------------

#Se extraen los datos de la matriz G
Gfile=open("G324.txt", "r")
Gstr=Gfile.read()
Gstrsplit=Gstr.split()

#Se extraen los datos de las estaciones
dfile=open("d.txt", "r")
dstr=dfile.read()
dstrsplit=dstr.split()

#Se define un arreglo para la matriz G
G=np.zeros((Nd,Nm))
#Se define un arreglo para los datos
d=np.zeros((Nd,1))

#Se almacenan los datos de la matriz G y el vector d en los arreglos correspondientes
for i in range (0,Nd):
    G[i,:]=np.array(Gstrsplit[(i*(Nm)):(i*(Nm))+Nm])
    d[i][0]=float(dstrsplit[i])
    
#Se define la transpuesta de la matriz G   
GT=np.transpose(G)

In [10]:
#----------------------------Matriz de covarianza de los datos----------------------------
#Se definen los valores de varianza de cada componente por estación
Vd=[2.1**2,2.5**2,5.1**2]*int(Nd/3)
#Se convierte la lista de valores a un arreglo numpy y las unidades a metros
Vd=np.array(Vd)*(10**-3)
#Se especifíca el vector de varianzas como vector columna
Vd=Vd.reshape(Nd,1)
#Se define una matriz cuya diagonal corresponde al vector de varianzas para obtener la matriz de covarianza de los datos
Cd=Vd*np.identity(Nd)
#Se obtiene la inversa de la matriz de covarianza de los datos
Cdinv=np.linalg.inv(Cd)

In [11]:
#------------------------Espacio de parámetros------------------------

#Se define el límite inferior de los parámetros
liminf=-0.004931
#Se define el límite superior de los parámetros
limsup=4.931506

#Se define un vector con el límite inferior de cada parámetro
a=np.zeros((1,Nm))+liminf
#Se define un vector con el límite superior de cada parámetro
b=np.zeros((1,Nm))+limsup
#Se define el valor de alpha
alpha=8
#Se define un vector con los valores posibles para un parámetro
mn=np.linspace(liminf,limsup,Ndm)
#Se define una matriz que representa el espacio de parámetros
m=np.zeros((len(mn),Nm))+mn.reshape(Ndm,1)
#Se define un vector con la media de los parámetros (pp 372, primer parrafo, tercer renglón)
m0=np.zeros((Nm,1))+(limsup/2)

In [12]:
#Se calcula el vector de medias y la matriz de covarianza de la posterior
mv_c,Cmv_c=mvCmv(G,d,Cdinv,alpha,limsup,m0)

In [ ]:
#Se define una semilla de números aleatorios para generar resultados reproducibles
np.random.seed(seed=10)
#Se define el número de muestras que se van a simular 
SampleSize=int(1400000)
start_MCMC=time.perf_counter() 
#Se simulan las muestras de la posterior
TruncadaCS324=SimulaTruncada(mv_c,Cmv_c,liminf,limsup,SampleSize)
end_MCMC=time.perf_counter()
#Se calcula el tiempo de ejecución con el método MCMC
t_MCMC=end_MCMC-start_MCMC
print(t_MCMC)